# Librerias y datos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
pd.options.mode.chained_assignment = None  # default='warn'

path = "data/ARIA_merged&selected/"

# planes:
pat = pd.read_csv(path + "Patient_real.csv")
cur = pd.read_csv(path + "Course_pi_diag.csv")
rtp = pd.read_csv(path + "RTPlan_PS_Pres.csv")
rad_grp = pd.read_csv(path + "Rad_grp.csv")

# citas:
sch = pd.read_csv(path + "SchActivity.csv")
sch_plan = pd.read_csv(path + "Sch_Plan.csv")
tr = pd.read_csv(path + "TreatmentRecord.csv")

res = ut.leer_csv("data/ARIA/12.Machine/z_Machine.csv", 1)

Archivo: data/ARIA/12.Machine/z_Machine.csv
Se dejaron 14 columnas.
Se eliminaron 6 columnas:




# Planes
### Join

In [6]:
planes = rtp\
.merge(cur, on="CourseSer", how="inner")\
.merge(pat, on="PatientSer", how="inner")\
.merge(rad_grp, on="PlanSetupSer", how='inner')
print(planes.shape)
planes = planes[planes.RTPlanSer.isin(tr.RTPlanSer.unique())]
print(planes.shape)

(13494, 51)
(8192, 51)


### Limpieza

In [3]:
c_index = "RTPlanSer"
c_elim = ["PlanSetupSer","SeriesSer","CourseSer","PatientSer","PrescriptionSer"]
c_fechas = ["StartDateTime"]
c_num = []
c_nan_bin = []

nun_th = 30
len_th = 30
num_div = 6

planes2, c_categ, elim, res = ut.limpiar_tabla(planes, c_index, c_elim, c_fechas, c_num, c_nan_bin, nun_th, len_th, num_div)

In [4]:
res.sort_values(by="Tipo")

,Tipo,nans,NumCategorias,NumValoresAgrupados,FrecRelValoresAgrupados
ClinicalStatus,categorica,0.000000,2,NaN,NaN
CityOrTownship,categorica,1.599121,31,596.000000,0.326782
Notes,categorica,87.487793,31,524.000000,0.085571
Site,categorica,63.024902,31,87.000000,0.054565
PlanNormMethod,categorica,59.020996,31,167.000000,0.027100
HistologyCode,categorica,74.206543,31,91.000000,0.026001
TStage,categorica,46.044922,31,49.000000,0.020630
CalcModelOptions,categorica,61.157227,31,45.000000,0.009521
DiagnosisCode_sec,categorica,90.661621,31,4.000000,0.000488
NStage,categorica,60.974121,29,NaN,NaN


### Guardo el dataset

In [5]:
planes2.to_csv("data/ARIA_cleaned/planes_cat.csv", index=True)

# Citas
### join

In [5]:
sch["ScheduledStartTime"] = pd.to_datetime(sch["ScheduledStartTime"])
sch["ScheduledEndTime"] = pd.to_datetime(sch["ScheduledEndTime"])
sch["ActualStartDate"] = pd.to_datetime(sch["ActualStartDate"])
sch["ActualEndDate"] = pd.to_datetime(sch["ActualEndDate"])

In [9]:
citas = sch[(sch.PatientSer.isin(pat.PatientSer)) & (sch.ActualEndDate.notnull()) 
           & (sch.ActualStartDate.notnull()) & (sch.ActualStartDate != sch.ActualEndDate)]

citas = citas\
.merge(sch_plan, on="ScheduledActivitySer", how="inner")\
.merge(planes, on="RTPlanSer", how="inner", suffixes = ("_sch", "_planes"))
citas = citas.sort_values(by="ActualStartDate")

citas["ScheduledDuration"] = (citas.ScheduledEndTime - citas.ScheduledStartTime).astype("int64")*1e-9/60
citas["ActualDuration"] = (citas.ActualEndDate - citas.ActualStartDate).astype("int64")*1e-9/60
citas = citas[(citas.ActualDuration>1) & (citas.ActualDuration<300)]
citas["NumPlans"] = citas.groupby('ScheduledActivitySer')['ScheduledActivitySer'].transform('count')
citas["DayOfWeek"] = citas.ActualStartDate.dt.dayofweek
citas["IsMorning"] = citas.ActualStartDate.dt.hour<15
citas["TimesHasCome"] = citas.groupby('RTPlanSer').cumcount()
print(citas.shape)

(76881, 92)


### Limpieza

In [16]:
c_index = "ScheduledActivitySer"
c_elim = ['PatientSer_sch','DiagnosisSer','RTPlanSer','SeriesSer','PlanSetupSer','PrescriptionSer','PatientSer_planes',
          'CourseSer_sch','CourseSer_planes','NoOfFractions']
c_fechas = ['StartDateTime',"ScheduledStartTime","ScheduledEndTime","ActualStartDate","ActualEndDate"]
c_num = ['DefaultDuration','ScheduledDuration','Duration','PrescribedPercentage','DateOfBirth',
         'PrescribedDose','NumFields','StartDelay','NumPlans','TimesHasCome']
# c_num = []
c_nans_bin = []

nun_th = 30
len_th = 30
num_div = -1

citas2, c_categ, elim, res = ut.limpiar_tabla(citas, c_index, c_elim, c_fechas, c_num, c_nans_bin, nun_th, len_th,num_div)
citas2.shape

(76881, 80)

In [17]:
res.sort_values(by="nans")[:60]

,Tipo,nans,NumCategorias,NumValoresAgrupados,FrecRelValoresAgrupados
ScheduledActivityCode,categorica,0.000000,2.0,NaN,NaN
ActualMachineSer,num -> categ,0.000000,4.0,NaN,NaN
NotificationPriorTime_sch,num -> categ,0.000000,4.0,NaN,NaN
NotificationPriorTime_a,num -> categ,0.000000,3.0,NaN,NaN
StartDelay_isNaN,num -> categ,0.000000,2.0,NaN,NaN
PrescribedDose_isNaN,num -> categ,0.000000,2.0,NaN,NaN
DateOfBirth_isNaN,num -> categ,0.000000,2.0,NaN,NaN
DicomSeqNumber,num -> categ,0.000000,4.0,NaN,NaN
IsMorning,num -> categ,0.000000,2.0,NaN,NaN
MultiFieldOptFlag,num -> categ,0.000000,2.0,NaN,NaN


### Columnas categoricas

In [18]:
print(c_categ)

['ScheduledActivityCode', 'CityOrTownship', 'NStage', 'Technique', 'NoEditFlag_sch', 'NotificationPriorTimeFlag_sch', 'Site', 'NumberOfFractions_isNaN', 'InterfaceStamp', 'Intent', 'IsMorning', 'TechniqueSer', 'TStage', 'TumorSize_isNaN', 'SchedulableFlag', 'SimulationNeeded', 'DayOfWeek', 'NotificationPriorTime_a', 'SummaryStage', 'TreatmentIntentType', 'Sex', 'NotificationPriorTimeFlag_a', 'DiagnosisCode_sec', 'MStage', 'ActivityCategoryCode', 'NotificationPriorTime_sch', 'ObjectStatus_sec', 'Description', 'ResourceGroupSer', 'ActivityCode', 'FractionFrequency', 'ObjectStatus_planes', 'PlanNormMethod', 'ActualMachineSer', 'AssignableFlag', 'Status_pres', 'ClinicalStatus', 'PhaseType', 'PrimaryPTVSer_isNaN', 'SkinFlashMargin', 'PrescribedDose_isNaN', 'HistologyCode', 'DateOfBirth_isNaN', 'Status_rtpps', 'StartDelay_isNaN', 'CalcModelOptions', 'EquipmentSer', 'Bolus', 'AnchorActivityFlag', 'DiagnosisCode', 'DicomSeqNumber', 'NoEditFlag_cat', 'MultiFieldOptFlag', 'TreatmentOrientation',

### Guardo el dataset

In [19]:
citas2.to_csv("data/ARIA_cleaned/citas.csv", index=True)